In [1]:
# Importamos librerias
import pandas as pd
import numpy as np

from datetime import datetime 

import os

import re

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# Ignorar warings
import warnings
warnings.filterwarnings("ignore")

import dotenv
from dotenv import load_dotenv
load_dotenv()


pd.set_option('display.max_columns', None)

import sys 
sys.path.append(os.path.abspath("../src"))  

In [2]:
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors 

from psycopg2 import sql
import support_base_de_datos as sbd
import support_extraccion as se


Para la creacion de tablas, he decidido hacer 3 tablas una para para las ediciones de las grandes vueltas, otra para las etapas de cada edición y otra para los puertos más frecuents. He tanteado la idea de cada una de estas tablas dividirla en 3 una para el giro, ota para el Tour y otra para la Vuela, pero eso complicaría bastante a la hora de hacer las queries así que lo he desechado.

 

En primer lugar cargamos los datafrmaes en los que basaremos nuestra inserción, que serán los 3 de información general delas carreras, los 3 de información de etapas y los 3 de información de puertos.

In [3]:
dicc_dataframes=se.cargar_datos_wikipedia()
[df_giro, df_tour, df_vuelta] = [dicc_dataframes["datos_giro"],
                                 dicc_dataframes["datos_tour"] ,
                                 dicc_dataframes["datos_vuelta"]]

In [4]:
df_giro.head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de corredores que participaron,N.º de corredores que finalizaron,Puntos del ganador
0,1,1909,Luigi Ganna,NaN,2448.0,27.258,127,49,25
1,2,1910,Carlo Galetti,NaN,2987.5,26.114,101,22,28
2,3,1911,Carlo Galetti,NaN,3452.0,26.082,86,24,50
3,4,1912,Atala,NaN,2439.5,27.106,54,26,31
4,5,1913,Carlo Oriani,NaN,2932.0,26.379,99,35,37


In [5]:
lista_carreras=["giro-d-italia", "tour-de-france", "vuelta-a-espana"]

In [6]:
for carrera in lista_carreras:
    ruta_datos = '../datos/datos_api_procyclingstats/datos_limpiados'
    nombre_archivo = f'df_etapas_{carrera}.csv'
    ruta_archivo = os.path.join(ruta_datos, nombre_archivo)
    if carrera == "giro-d-italia":
        df_etapas_giro=pd.read_csv(ruta_archivo)
    elif carrera == "tour-de-france":
        df_etapas_tour=pd.read_csv(ruta_archivo)
    elif carrera == "vuelta-a-espana":
        df_etapas_vuelta=pd.read_csv(ruta_archivo)

In [7]:
df_etapas_giro.head()

,arrival,avg_speed_winner,date,departure,distance,stage_type,vertical_meters,rider_name,rider_url,age,nationality,race_name
0,Bologna,28.15,1909-05-13,Milano,397.0,RR,NaN,BENI Dario,https://www.procyclingstats.com/rider/dario-beni,20,IT,giro-d-italia
1,Napoli,25.56,1909-05-18,Chieti,242.0,RR,NaN,ROSSIGNOLI Giovanni,https://www.procyclingstats.com/rider/giovanni...,26,IT,giro-d-italia
2,Roma,26.72,1909-05-20,Napoli,228.0,RR,NaN,GANNA Luigi,https://www.procyclingstats.com/rider/luigi-ganna,25,IT,giro-d-italia
3,Firenze,27.00,1909-05-23,Roma,346.0,RR,NaN,GANNA Luigi,https://www.procyclingstats.com/rider/luigi-ganna,25,IT,giro-d-italia
4,Genua,26.85,1909-05-25,Firenze,294.0,RR,NaN,ROSSIGNOLI Giovanni,https://www.procyclingstats.com/rider/giovanni...,26,IT,giro-d-italia


In [8]:
df_puertos_giro=pd.read_csv('../datos/datos_api_geopy/datos_transformados/datos_puertos_giro.csv')
df_puertos_tour=pd.read_csv('../datos/datos_api_geopy/datos_transformados/datos_puertos_tour.csv')
df_puertos_vuelta=pd.read_csv('../datos/datos_api_geopy/datos_transformados/datos_puertos_vuelta.csv')

In [9]:
df_puertos_giro.head()

,Unnamed: 0,Climb,#stages,#editions,First year,Latitud,Longitud
0,0,Roccaraso,19,14,1952,41.850261,14.078257
1,1,Passo dell'Aprica,18,15,1962,46.154330,10.154179
2,2,Passo Pordoi,16,16,1940,46.487552,11.812193
3,3,Passo del Mortirolo,13,13,1994,46.247859,10.298304
4,4,Passo dello Stelvio,12,12,1956,46.548023,10.505531


Para nuestra inserción de tablas y datos nos interesa agregar a los dataframes de etapas de giro, tour y vuelta un columna edition_id de numeración acumulativa entre los dataframes (es decir que si empezamos por df_etapas_giro, en df_etapas_tour la numeración de esa columna sigue a partir del ultimo valor que tomo en el dataframe anterior, como si estuvieran concatenados).

Ciertamente parecería mas natural concatenar los dataframes, hay dos motivos para no hacerlo. Primero, inicalmente como se dijo más arriba la idea era de crear una tabla por carrera, y en eso hemos basado la estructura de los datos extraídos, y ahora simplemente la vamos a usar para insertar los datos juntos de las tres carreras, pues basta simplemente juntar los dataframes en una lista y usar for, maps o zips segun el caso para iterar sobre dicha lista. La segunda razón y más fundamental, el archivo de datos ya pesa mucho habiendome visto obligado a comprimrlo para poder subir el repositorio a github. Los datos de las etapas extraidos de la api de procyclingstats son con diferencia los más pesaeos, crear un nuevo archivo con toda la información de todas las etapas crea serio riesgo de no poder subir el repositorio incluso si se comprime la carpeta de datos.

Para que este edition_id que sera una foreign key proveniente de la tabla ediciones este correctamente definido en relacion con dicha tabla de ediciones tenemos que tener en cuenta que en nuestros dataframes de etapas faltan a veces datos de años enteros, pues la api de procyclingstats no consiguio devolver información para todas las ediciones de todas las carreras. Es por esto, que añadimos una fila de nulos, a cada dataframe de etapas por cada edición de la carrera de la que no haya información. Ojo, lo hacemos solo si no hay información de ninguna etapa, no si falta información de alguna. Esto lo hacemos para que la columna edition_id lleve una numeración correlativa a la del id de la tabla ediciones. Y precisamente por eso, esta columna nos interesa que nunca sea nula, por lo que el proceso de añadir nulos lo haremos antes de insertar dicha columna.



def insertar_filas_nulas(df,rangos_exclusion):
    """
    Inserta filas con valores nulos en un DataFrame para rellenar huecos de años mayores o iguales a 2 en la columna 'date',
    excluyendo los años especificados en rangos_exclusion.
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame que debe contener una columna 'date' en formato datetime para identificar los huecos de año.
    
    rangos_exclusion : list of tuple, optional
        Lista de tuplas que representan los rangos de años a excluir. Cada tupla contiene el inicio y fin de un rango (inclusive).
    
    Returns
    -------
    pandas.DataFrame
        DataFrame con las filas nulas insertadas según las condiciones especificadas.
    """
    df = df.reset_index(drop=True) 
    df['date'] = pd.to_datetime(df['date'])
    
    filas_a_insertar = []
    for i in range(1, len(df)):
        anio_actual = df.loc[i, 'date'].year
        anio_anterior = df.loc[i - 1, 'date'].year
        diferencia_anios = anio_actual - anio_anterior
        
        # Comprobamos que la diferencia es mayor o igual a 2 (pues eso es lo que indica que falta una edición entera)
        #  y que no estamos dentro en los rangos de exclusión
        if diferencia_anios >= 2 and not any(inicio <= anio_actual <= fin for inicio, fin in rangos_exclusion):
            num_filas_nulas = diferencia_anios - 1
            for j in range(1, num_filas_nulas + 1):
                fila_nula = df.loc[i - 1].copy()  # Copiamos la fila anterior para obtener el esquema
                fila_nula[:] = None  # Asignamos valores nulos a toda la fila
                fila_nula['date'] = pd.to_datetime(f"{anio_anterior + j}-01-01")  # Ajustamos la fecha a un año intermedio
                filas_a_insertar.append((i + len(filas_a_insertar), fila_nula))
    
    # Insertar las filas nulas en el DataFrame
    for indice, fila in filas_a_insertar:
        df = pd.concat([df.iloc[:indice], pd.DataFrame([fila]), df.iloc[indice:]]).reset_index(drop=True)

    return df



In [10]:
def insertar_filas_nulas(df):
    """
    Inserta filas con valores nulos en un DataFrame para rellenar huecos de años mayores o iguales a 2 en la columna 'date'.
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame que debe contener una columna 'date' en formato datetime para identificar los huecos de año.
    
    Returns
    -------
    pandas.DataFrame
        DataFrame con las filas nulas insertadas según las condiciones especificadas.
    """
    df = df.reset_index(drop=True) 
    df['date'] = pd.to_datetime(df['date'])
    
    filas_a_insertar = []
    for i in range(1, len(df)):
        anio_actual = df.loc[i, 'date'].year
        anio_anterior = df.loc[i - 1, 'date'].year
        diferencia_anios = anio_actual - anio_anterior
        
        # Comprobamos que la diferencia es mayor o igual a 2 (pues eso es lo que indica que falta una edición entera)
        #  y que no estamos dentro en los rangos de exclusión
        if diferencia_anios >= 2:
            num_filas_nulas = diferencia_anios - 1
            for j in range(1, num_filas_nulas + 1):
                fila_nula = df.loc[i - 1].copy()  # Copiamos la fila anterior para obtener el esquema
                fila_nula[:] = None  # Asignamos valores nulos a toda la fila
                fila_nula['date'] = pd.to_datetime(f"{anio_anterior + j}-01-01")  # Ajustamos la fecha a un año intermedio
                filas_a_insertar.append((i + len(filas_a_insertar), fila_nula))
    
    # Insertar las filas nulas en el DataFrame
    for indice, fila in filas_a_insertar:
        df = pd.concat([df.iloc[:indice], pd.DataFrame([fila]), df.iloc[indice:]]).reset_index(drop=True)

    return df



In [11]:
def filtrar_por_rango_anios(df, rangos_anios):
    """
    Elimina las filas de un DataFrame donde el año en la columna 'date' pertenece a un rango específico.

    Args:
        df (pd.DataFrame): El DataFrame a procesar. Debe tener una columna 'date' en formato de fecha o string.
        rangos_anios (list of tuples): Lista de tuplas que representan los rangos de años (inicio, fin).

    Returns:
        pd.DataFrame: El DataFrame filtrado sin las filas que corresponden a los rangos de años especificados.
    """
    # Asegurarse de que la columna 'date' esté en formato datetime
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # Iterar sobre los rangos y eliminar los años dentro de esos rangos
    for inicio, fin in rangos_anios:
        df = df[~df['date'].dt.year.between(inicio, fin)]

    return df




rangos_exclusion_giro = [(1915, 1918), (1941, 1945)]

df_etapas_giro_filtrado = filtrar_por_rango_anios(df_etapas_giro, rangos_exclusion_giro)

In [12]:
rangos_exclusion_giro = [(1915, 1918), (1941, 1945)]
df_etapas_giro = insertar_filas_nulas(df_etapas_giro)
df_etapas_giro = filtrar_por_rango_anios(df_etapas_giro, rangos_exclusion_giro)

rangos_exclusion_tour = [(1915, 1918), (1940, 1946)]
df_etapas_tour = insertar_filas_nulas(df_etapas_tour,rangos_exclusion_tour)

In [13]:
rangos_exclusion_tour = [(1915, 1918), (1940, 1946)]
df_etapas_tour = insertar_filas_nulas(df_etapas_tour)
df_etapas_tour = filtrar_por_rango_anios(df_etapas_tour,rangos_exclusion_tour)

rangos_exclusion_vuelta = [(1937, 1940), (1943, 1944),(1949,1949),(1951,  1954)]
df_etapas_vuelta = insertar_filas_nulas(df_etapas_vuelta,rangos_exclusion_vuelta)

In [14]:
rangos_exclusion_vuelta = [(1937, 1940), (1943, 1944),(1949,1949),(1951, 1954)]
df_etapas_vuelta = insertar_filas_nulas(df_etapas_vuelta)
df_etapas_vuelta = filtrar_por_rango_anios(df_etapas_vuelta,rangos_exclusion_vuelta)

In [15]:
def agregar_edition_id_acumulativo(df, start_value=1):
    """
    Agrega una columna 'edition_id' que aumenta cada vez que hay un cambio de año en el DataFrame,
    y acumula la numeración a partir de un valor inicial especificado.
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame al que se añadirá la columna 'edition_id', basada en cambios de año en la columna 'date'.
        
    start_value : int, default=1
        El valor inicial para comenzar la numeración de 'edition_id'.

    Returns
    -------
    df : pandas.DataFrame
        DataFrame con la columna 'edition_id' añadida.
    
    last_value : int
        Último valor de 'edition_id' en el DataFrame después de aplicar la función, incrementado en 1 para el siguiente DataFrame.
    """
    df['anio'] = pd.to_datetime(df['date']).dt.year  
    df['edition_id'] = start_value + df['anio'].ne(df['anio'].shift()).cumsum() - 1  # Calculamos el edition_id acumulativo
    df.drop(columns='anio', inplace=True)  # Eliminamos la columna auxiliar 'anio'
    last_value = df['edition_id'].iloc[-1] + 1  # Último valor + 1 para el siguiente DataFrame
    return df, last_value





In [16]:
# Aplicamos la función a los 3 dataframe de etapas
dataframes_etapas = [df_etapas_giro, df_etapas_tour, df_etapas_vuelta]
start_value = 1
for i, df in enumerate(dataframes_etapas):
    dataframes_etapas[i], start_value = agregar_edition_id_acumulativo(df, start_value)

Comprobamos que edition_id se ha agregado como deseamos.

In [17]:
df_etapas_giro.head(1)


,arrival,avg_speed_winner,date,departure,distance,stage_type,vertical_meters,rider_name,rider_url,age,nationality,race_name,edition_id
0,Bologna,28.15,1909-05-13,Milano,397.0,RR,NaN,BENI Dario,https://www.procyclingstats.com/rider/dario-beni,20,IT,giro-d-italia,1


In [18]:
df_etapas_giro.tail(1)

,arrival,avg_speed_winner,date,departure,distance,stage_type,vertical_meters,rider_name,rider_url,age,nationality,race_name,edition_id
1227,Rome,43.647,2024-05-26,Rome,125.0,RR,676.0,MERLIER Tim,https://www.procyclingstats.com/rider/tim-merlier,31,BE,giro-d-italia,107


In [19]:
df_etapas_tour.head(1)

,arrival,avg_speed_winner,date,departure,distance,stage_type,vertical_meters,rider_name,rider_url,age,nationality,race_name,edition_id
0,Lyon,26.3,1903-07-01,Montgeron,467.0,RR,NaN,GARIN Maurice,https://www.procyclingstats.com/rider/maurice-...,32.0,FR,tour-de-france,108


In [20]:
df_etapas_tour.tail(1)

,arrival,avg_speed_winner,date,departure,distance,stage_type,vertical_meters,rider_name,rider_url,age,nationality,race_name,edition_id
2387,Nice,44.521,2024-07-21,Monaco,33.7,ITT,720.0,POGAČAR Tadej,https://www.procyclingstats.com/rider/tadej-po...,25.0,SI,tour-de-france,218


In [21]:
df_etapas_vuelta.head(1)

,arrival,avg_speed_winner,date,departure,distance,stage_type,vertical_meters,rider_name,rider_url,age,nationality,race_name,edition_id
0,Valladolid,30.99,1935-04-29,Madrid,185.0,RR,NaN,DIGNEF Antoon,https://www.procyclingstats.com/rider/antoon-d...,24.0,BE,vuelta-a-espana,219


In [22]:
df_etapas_vuelta.tail(1)

,arrival,avg_speed_winner,date,departure,distance,stage_type,vertical_meters,rider_name,rider_url,age,nationality,race_name,edition_id
1643,Madrid,55.768,2024-09-08,Madrid,24.6,ITT,190.0,KÜNG Stefan,https://www.procyclingstats.com/rider/stefan-kung,30.0,CH,vuelta-a-espana,297


### Creamos la base de datos y la conexion

In [23]:
sbd.crear_basedatos("localhost","postgres","admin","5432","Ciclismo")

Error de tipo: la base de datos «Ciclismo» ya existe



In [24]:
# Creamos la conexion
conn = psycopg2.connect(
    dbname="Ciclismo",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()


### Creamos las tablas

Sobre la relación entre las tablas, la tabla de ediciones y la de etapas están conectadas mediante id_ediciones que es Primary Key de la tabla ediciones y foreign key de la tabla etapas. En cambio, para la tabla de puertos, dado que no tenemos información sobre en que edición se subió cada uno no hemso podido conectarlo con una primary key, sin embargo, el campo carrera conecta las 3 tablas lo que permite hacer consultas.

In [25]:
cursor.execute("""
-- Tabla para las ediciones de las grandes vueltas
CREATE TABLE IF NOT EXISTS ediciones (
    id SERIAL PRIMARY KEY,
    carrera VARCHAR(20),
    edicion INT,
    ano INT,
    ganador VARCHAR(100),
    tiempo_ganador VARCHAR(50),
    kilometros_totales FLOAT,
    velocidad_media FLOAT,
    num_corredores_participantes INT,
    num_corredores_finalizaron INT,
    puntos_ganador INT
);

-- Tabla para las etapas de cada edición (sin el campo 'etapa')
CREATE TABLE IF NOT EXISTS etapas (
    id SERIAL PRIMARY KEY,
    carrera VARCHAR(20),
    fecha DATE,
    salida VARCHAR(100),
    llegada VARCHAR(100),
    distancia FLOAT,
    tipo_etapa VARCHAR(10),
    metros_verticales FLOAT,
    ganador_etapa VARCHAR(100),
    url_ganador VARCHAR(255),
    edad INT,
    nacionalidad VARCHAR(10),
    edicion_id INT REFERENCES ediciones(id) ON DELETE CASCADE
);

-- Tabla para los puertos más frecuentados
CREATE TABLE IF NOT EXISTS puertos (
    id SERIAL PRIMARY KEY,
    carrera VARCHAR(20),
    puerto VARCHAR(100),
    num_etapas INT,
    num_ediciones INT,
    ano_primera_vez INT,
    latitud FLOAT,
    longitud FLOAT
);
""")

conn.commit()

### Inserción de datos

Usamos el metodo iterrows para iterar por filas. Hacemos las inserciones fijandonos en respetar la primary key y que los values se corresponden a los campos que queremos meter.

for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_giro, df_tour, df_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO ediciones (carrera, edicion, ano, ganador, tiempo_ganador, kilometros_totales, velocidad_media,
                                   num_corredores_participantes, num_corredores_finalizaron, puntos_ganador)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            int(row['Edición']),
            int(row['Año']),
            row['Ganador'],
            row['Tiempo del ganador'] if pd.notna(row['Tiempo del ganador']) else None,
            row['Kilómetros totales'] if pd.notna(row['Kilómetros totales']) else None,
            row['Velocidad media'] if pd.notna(row['Velocidad media']) else None,
            int(row['N.º de corredores que participaron']) if pd.notna(row['N.º de corredores que participaron']) else None,
            int(row['N.º de corredores que finalizaron']) if pd.notna(row['N.º de corredores que finalizaron']) else None,
            int(row['Puntos del ganador']) if pd.notna(row['Puntos del ganador']) else None
        ))

# Recuperar los IDs de las ediciones insertadas para usarlos en las etapas
cursor.execute("SELECT id, carrera, edicion FROM ediciones")
ediciones_ids = cursor.fetchall()
ediciones_map = {(carrera, edicion): id for id, carrera, edicion in ediciones_ids}


# Inserción en etapas (modificado para mapear edicion_id correctamente)
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_etapas_giro, df_etapas_tour, df_etapas_vuelta]):
    for _, row in df.iterrows():
        # Mapea edicion_id usando ediciones_map
        edicion_id = ediciones_map.get((carrera, row['edition_id']))
        if edicion_id:
            cursor.execute("""
                INSERT INTO etapas (carrera, fecha, salida, llegada, distancia, tipo_etapa, metros_verticales,
                                    ganador_etapa, url_ganador, edad, nacionalidad, edicion_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                carrera,
                row['date'],
                row['departure'],
                row['arrival'],
                row['distance'] if pd.notna(row['distance']) else None,
                row['stage_type'],
                row['vertical_meters'] if pd.notna(row['vertical_meters']) else None,
                row['rider_name'],
                row['rider_url'],
                int(row['age']) if pd.notna(row['age']) else None,
                row['nationality'],
                edicion_id
            ))
# Insertar datos en puertos
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_puertos_giro, df_puertos_tour, df_puertos_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO puertos (carrera, puerto, num_etapas, num_ediciones, ano_primera_vez, latitud, longitud)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            row['Climb'],
            int(row['#stages']),
            int(row['#editions']),
            int(row['First year']),
            row['Latitud'] if pd.notna(row['Latitud']) else None,
            row['Longitud'] if pd.notna(row['Longitud']) else None
        ))
conn.commit()

for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_giro, df_tour, df_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO ediciones (carrera, edicion, ano, ganador, tiempo_ganador, kilometros_totales, velocidad_media,
                                   num_corredores_participantes, num_corredores_finalizaron, puntos_ganador)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            int(row['Edición']),
            int(row['Año']),
            row['Ganador'],
            row['Tiempo del ganador'] if pd.notna(row['Tiempo del ganador']) else None,
            row['Kilómetros totales'] if pd.notna(row['Kilómetros totales']) else None,
            row['Velocidad media'] if pd.notna(row['Velocidad media']) else None,
            int(row['N.º de corredores que participaron']) if pd.notna(row['N.º de corredores que participaron']) else None,
            int(row['N.º de corredores que finalizaron']) if pd.notna(row['N.º de corredores que finalizaron']) else None,
            int(row['Puntos del ganador']) if pd.notna(row['Puntos del ganador']) else None
        ))

# Recuperar los IDs de las ediciones insertadas para usarlos en las etapas
cursor.execute("SELECT id, carrera, edicion FROM ediciones")
ediciones_ids = cursor.fetchall()
ediciones_map = {(carrera, edicion): id for id, carrera, edicion in ediciones_ids}

# Inserción en etapas
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_etapas_giro, df_etapas_tour, df_etapas_vuelta]):
    for _, row in df.iterrows():
        # Mapea edicion_id usando ediciones_map
        edicion_id = ediciones_map.get((carrera, row['edition_id']))
        print(edicion_id)
        if edicion_id is None:
            print(f"Advertencia: No se encontró edicion_id para carrera {carrera} y edición {row['edition_id']}")
            continue  # Saltar filas con errores

        try:
            cursor.execute("""
                INSERT INTO etapas (carrera, fecha, salida, llegada, distancia, tipo_etapa, metros_verticales,
                                    ganador_etapa, url_ganador, edad, nacionalidad, edicion_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                carrera,
                row['date'],
                row['departure'],
                row['arrival'],
                row['distance'] if pd.notna(row['distance']) else None,
                row['stage_type'],
                row['vertical_meters'] if pd.notna(row['vertical_meters']) else None,
                row['rider_name'],
                row['rider_url'],
                int(row['age']) if pd.notna(row['age']) else None,
                row['nationality'],
                edicion_id
            ))
        except Exception as e:
            print(f"Error al insertar etapa para carrera {carrera}: {e}")


# Insertar datos en puertos
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_puertos_giro, df_puertos_tour, df_puertos_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO puertos (carrera, puerto, num_etapas, num_ediciones, ano_primera_vez, latitud, longitud)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            row['Climb'],
            int(row['#stages']),
            int(row['#editions']),
            int(row['First year']),
            row['Latitud'] if pd.notna(row['Latitud']) else None,
            row['Longitud'] if pd.notna(row['Longitud']) else None
        ))
conn.commit()
cursor.close()

In [26]:
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_giro, df_tour, df_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO ediciones (carrera, edicion, ano, ganador, tiempo_ganador, kilometros_totales, velocidad_media,
                                   num_corredores_participantes, num_corredores_finalizaron, puntos_ganador)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            int(row['Edición']),
            int(row['Año']),
            row['Ganador'],
            row['Tiempo del ganador'] if pd.notna(row['Tiempo del ganador']) else None,
            row['Kilómetros totales'] if pd.notna(row['Kilómetros totales']) else None,
            row['Velocidad media'] if pd.notna(row['Velocidad media']) else None,
            int(row['N.º de corredores que participaron']) if pd.notna(row['N.º de corredores que participaron']) else None,
            int(row['N.º de corredores que finalizaron']) if pd.notna(row['N.º de corredores que finalizaron']) else None,
            int(row['Puntos del ganador']) if pd.notna(row['Puntos del ganador']) else None
        ))

# Crear un mapeo basado en carrera y edición
cursor.execute("SELECT id, carrera, edicion FROM ediciones")
lista_tuplas_ids_carreras = cursor.fetchall()

# Crear un mapeo donde la clave es (carrera, edición)
ediciones_map = {(carrera, edicion): id for id, carrera, edicion in lista_tuplas_ids_carreras}

# Inserción en etapas
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_etapas_giro, df_etapas_tour, df_etapas_vuelta]):
    for _,row in df.iterrows():
        # Convertir row.edition_id a edición local para la carrera actual
        edicion_local = row["edition_id"] - df['edition_id'].min() + 1

        # Obtener edicion_id basado en carrera y edición local
        edicion_id = ediciones_map.get((carrera, edicion_local))
        cursor.execute("""
            INSERT INTO etapas (carrera, fecha, salida, llegada, distancia, tipo_etapa, metros_verticales,
                                ganador_etapa, url_ganador, edad, nacionalidad, edicion_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            row["date"],
            row["departure"],
            row["arrival"],
            row["distance"] if pd.notna(row["distance"]) else None,
            row["stage_type"],
            row["vertical_meters"] if pd.notna(row["vertical_meters"]) else None,
            row["rider_name"],
            row["rider_url"],
            int(row["age"]) if pd.notna(row["age"]) else None,
            row["nationality"],
            edicion_id
        ))
        

# Insertar datos en puertos
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_puertos_giro, df_puertos_tour, df_puertos_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO puertos (carrera, puerto, num_etapas, num_ediciones, ano_primera_vez, latitud, longitud)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            row['Climb'],
            int(row['#stages']),
            int(row['#editions']),
            int(row['First year']),
            row['Latitud'] if pd.notna(row['Latitud']) else None,
            row['Longitud'] if pd.notna(row['Longitud']) else None
        ))
conn.commit()
cursor.close()